In [19]:
import pandas as pd
import glob
import os
import numpy as np

# 범용 함수들

In [20]:
def raw_data_preprocessor(df_raw):
    df_result = df_raw.copy().dropna(axis= 0)
    df_result = drop_missing_rows(df_result)
    df_result = remove_string_whitespaces(df_result)
    df_result = drop_day_col(df_result)
    df_result = process_date(df_result)
    df_result = cast_float_to_int(df_result)
    df_result = replace_tilda_in_column(df_result)
    df_result = replace_on_off_bool(df_result)
    df_checker(df_result)
    return df_result

In [21]:
def drop_missing_rows(df, threshhold = 12):
    # 역번호 기준으로 필터링
    # 역명 기준으로 필터링한다.
    # 먼저 역번호 기준으로 평균 레코드 수는 730개 정도 된다.
    # 평균보다 현저히 작은 갯수의 레코드는 날린다.
    # 1년이 12일이니깐 이거보다 작은 데이터는 날린다.
    for station_number in df['역번호'].unique():
        if len(df[df['역번호'] == station_number]) < threshhold:
            print(station_number)
            df = df.copy().drop(df[df['역번호'] == station_number].index)
    for station_name in df['역명'].unique():
        if len(df[df['역명'] == station_name]) < threshhold:
            print(station_name)
            df = df.copy().drop(df[df['역명'] == station_name].index)
    return df

In [22]:
def process_date(df):
    if df['년월일'].dtype == 'object':
        df['년월일'] = df['년월일'].str.replace('-', '').replace('.', '')
        df['년월일'] = df['년월일'].copy().astype(int)
    return df
    

In [23]:
def df_checker(df):
    assert df.shape[0] > 30000
    assert df.shape[1] == 29
    assert sum(df.isna().sum()) == 0
    for col_name in df.columns:
        if df[col_name].dtype not in ['object', 'int32', 'bool']:  # added 'bool' to the list of accepted dtypes
            raise Exception(f'Invalid column dtype {df[col_name].dtype}')
    

In [24]:
def replace_tilda_in_column(df):
    for col_name in df.columns:
        if '~' in col_name:
            new_col_name = col_name.replace('~', '-')
            df[new_col_name] = df[col_name]
            df = df.drop(col_name, axis=1)
    return df

In [25]:
def replace_on_off_bool(df):
    df['구분'] = df['구분'].copy().map(lambda x : x == '승차')
    df['구분'] = df['구분'].astype(bool)
    print(df.dtypes)
    return df

In [26]:
def remove_string_whitespaces(df):
    for col_name in df.columns:
        if df[col_name].dtype == 'object':
            df[col_name] = df[col_name].map(lambda x : x.replace(' ', '').strip()).copy()
    return df
    

In [27]:
# 역번호, 년월일, 합계, 시점별 탑승량이 float으로 되어있으므로 int로 변경해준다.
# 결측치, NaN을 제거한 후 적용해야 한다.
def cast_float_to_int(df):
    for col_name in df.columns:
        if df[col_name].dtype == 'float' or df[col_name].dtype == 'int64':
            df[col_name] = df[col_name].astype('int32').copy()
    return df

In [28]:
def drop_day_col(df):
    try:
        df = df.copy().drop('요일', axis=1)
    except KeyError:
        # '요일이 없는 경우 KeyError 발생하는데, 이건 정상적인 에러'
        pass
    return df

In [29]:
# 모든 df 대상으로 범용함수 테스트해본다.
# 결과는 df_list에 담아 놓는다.
csv_files = glob.glob('../data/*.csv')
csv_files.sort()
df_list = []
for csv_file in csv_files:
    df_raw = pd.read_csv(csv_file, encoding='cp949')
    df_p = raw_data_preprocessor(df_raw)
    df_list.append(df_p)

401
역번호         int32
역명         object
년월일         int32
구분           bool
합계          int32
01시-02시     int32
02시-03시     int32
03시-04시     int32
04시-05시     int32
05시-06시     int32
06시-07시     int32
07시-08시     int32
08시-09시     int32
09시-10시     int32
10시-11시     int32
11시-12시     int32
12시-13시     int32
13시-14시     int32
14시-15시     int32
15시-16시     int32
16시-17시     int32
17시-18시     int32
18시-19시     int32
19시-20시     int32
20시-21시     int32
21시-22시     int32
22시-23시     int32
23시-24시     int32
24시-01시     int32
dtype: object
301.0
401.0
역번호         int32
역명         object
년월일         int32
구분           bool
합계          int32
01시-02시     int32
02시-03시     int32
03시-04시     int32
04시-05시     int32
05시-06시     int32
06시-07시     int32
07시-08시     int32
08시-09시     int32
09시-10시     int32
10시-11시     int32
11시-12시     int32
12시-13시     int32
13시-14시     int32
14시-15시     int32
15시-16시     int32
16시-17시     int32
17시-18시     int32
18시-19시     int32
19시-20시     int32
20시-21시     int3

In [30]:
df_list[0]['구분']

0         True
1        False
2         True
3        False
4         True
         ...  
76677    False
76678     True
76679    False
76680     True
76681    False
Name: 구분, Length: 76680, dtype: bool

In [31]:
# concat every dfs
def concat_melt_dfs(df_list):
    df_concated = pd.concat(df_list)
    df_melted = df_concated.melt(id_vars=['역번호', '역명', '년월일', '구분'], value_vars=['01시-02시', '02시-03시', '03시-04시',
        '04시-05시', '05시-06시', '06시-07시', '07시-08시', '08시-09시', '09시-10시',
        '10시-11시', '11시-12시', '12시-13시', '13시-14시', '14시-15시', '15시-16시',
        '16시-17시', '17시-18시', '18시-19시', '19시-20시', '20시-21시', '21시-22시',
        '22시-23시', '23시-24시', '24시-01시'])
    df_melted['hour'] = df_melted['variable'].str.slice(0, 2).astype(int)
    df_melted['timestamp'] = pd.to_datetime(df_melted['년월일'], format='%Y%m%d') + pd.to_timedelta(df_melted['hour'], unit='h')
    df_melted.drop(columns=['년월일', 'variable', 'hour'], inplace=True)
    df_melted.rename(columns={'value': 'passenger_count'}, inplace=True)


    df_melted.sort_values(['timestamp','역번호'], inplace=True)
    result = df_melted.reset_index()
    result = result.drop('index', axis=1)
    result.columns = ['id', 'name', 'on', 'traffic', 'timestamp']
    
    # timestamp를 인덱스로 지정해준다
    result.set_index('timestamp', inplace=True)
    
    # 동부산대 역이 윗반송으로 변경되었다.
    # 동부산대를 없애고 윗반송으로 대체해준다.
    result.loc[result['name']=='동부산대', 'name'] = '윗반송'
    assert result['name'].nunique() == result['id'].nunique(), f"두 컬럼의 고유값은 동일해야 한다. {result['name'].nunique()} {result['id'].nunique()}"
    return result


In [32]:
df_cleaned = concat_melt_dfs(df_list)

In [33]:
df_cleaned.head()

,id,name,on,traffic
timestamp,,,,
2014-01-01 01:00:00,101,신평,True,0
2014-01-01 01:00:00,101,신평,False,38
2014-01-01 01:00:00,102,하단,True,2
2014-01-01 01:00:00,102,하단,False,53
2014-01-01 01:00:00,103,당리,True,1


In [34]:
df_cleaned[df_cleaned['id'] == 102].loc['2022']

,id,name,on,traffic
timestamp,,,,
2022-01-01 00:00:00,102,하단,True,3
2022-01-01 00:00:00,102,하단,False,38
2022-01-01 01:00:00,102,하단,True,0
2022-01-01 01:00:00,102,하단,False,0
2022-01-01 02:00:00,102,하단,True,0
...,...,...,...,...
2022-12-31 21:00:00,102,하단,False,620
2022-12-31 22:00:00,102,하단,True,441
2022-12-31 22:00:00,102,하단,False,525


In [35]:
df_cleaned

,id,name,on,traffic
timestamp,,,,
2014-01-01 01:00:00,101,신평,True,0
2014-01-01 01:00:00,101,신평,False,38
2014-01-01 01:00:00,102,하단,True,2
2014-01-01 01:00:00,102,하단,False,53
2014-01-01 01:00:00,103,당리,True,1
...,...,...,...,...
2023-06-01 00:00:00,412,윗반송,False,27
2023-06-01 00:00:00,413,고촌,True,3
2023-06-01 00:00:00,413,고촌,False,4


In [55]:
df_cleaned['20230101'].name.unique()

/var/folders/mr/3m0tp4215cqg9hr952d47h8c0000gn/T/ipykernel_98007/2468673915.py:1: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  df_cleaned['20230101'].name.unique()


array(['다대포해수욕장', '다대포항', '낫개', '신장림', '장림', '동매', '신평', '하단', '당리', '사하',
       '괴정', '대티', '서대신', '동대신', '토성', '자갈치', '남포', '중앙', '부산역', '초량',
       '부산진', '좌천', '범일', '범내골', '1서면', '부전', '양정', '시청', '1연산', '교대',
       '1동래', '명륜', '온천장', '부산대', '장전', '구서', '두실', '남산', '범어사', '노포',
       '장산', '중동', '해운대', '동백', '벡스코', '센텀시티', '민락', '수영', '광안', '금련산',
       '남천', '경성대부경대', '대연', '못골', '지게골', '문현', '국제금융센터', '전포', '2서면',
       '부암', '가야', '동의대', '개금', '냉정', '주례', '감전', '사상', '덕포', '모덕', '모라',
       '구남', '구명', '2덕천', '수정', '화명', '율리', '동원', '금곡', '호포', '증산',
       '부산대양산', '남양산', '양산', '망미', '배산', '물만골', '3연산', '거제', '종합운동장',
       '사직', '미남', '만덕', '남산정', '숙등', '3덕천', '구포', '강서구청', '체육공원', '대저',
       '4동래', '수안', '낙민', '충렬사', '명장', '서동', '금사', '반여농산물', '석대', '영산대',
       '윗반송', '고촌', '안평'], dtype=object)

In [54]:
df_cleaned['20230101'].id.unique()

/var/folders/mr/3m0tp4215cqg9hr952d47h8c0000gn/T/ipykernel_98007/2931958184.py:1: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  df_cleaned['20230101'].id.unique()


array([ 95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107,
       108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120,
       121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133,
       134, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212,
       213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225,
       226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238,
       239, 240, 241, 242, 243, 302, 303, 304, 305, 306, 307, 308, 309,
       310, 311, 312, 313, 314, 315, 316, 317, 402, 403, 404, 405, 406,
       407, 408, 409, 410, 411, 412, 413, 414], dtype=int32)

In [63]:
selected = [1,2]
df_test = df_cleaned['20230101']
df_result = df_test.loc[(df_test['id'] // 100).isin(selected)]
df_result

/var/folders/mr/3m0tp4215cqg9hr952d47h8c0000gn/T/ipykernel_98007/1443485996.py:2: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  df_test = df_cleaned['20230101']


,id,name,on,traffic
timestamp,,,,
2023-01-01 00:00:00,100,동매,True,2
2023-01-01 00:00:00,100,동매,False,31
2023-01-01 00:00:00,101,신평,True,10
2023-01-01 00:00:00,101,신평,False,108
2023-01-01 00:00:00,102,하단,True,18
...,...,...,...,...
2023-01-01 23:00:00,241,부산대양산,False,24
2023-01-01 23:00:00,242,남양산,True,11
2023-01-01 23:00:00,242,남양산,False,70


In [65]:
import pickle
with open('../data/df_cleaned.pickle', 'wb') as f:
    pickle.dump(df_cleaned, f)